#### Gender Description
This script is to pull the gender for the CiC dataset.

In [ ]:
# Load libraries
library(dplyr)
library(here)
library(ggplot2)
library(bigrquery)
bq_auth()

In [ ]:
# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

# Store Tables of Interest
targetdb1 <-'yhcr-prd-phm-bia-core.CB_FDM_ChildrensSocialCare'
targetdb1 <-gsub(' ','',targetdb1)
print (targetdb1)

targetdb2 <-'yhcr-prd-phm-bia-core.CB_FDM_MASTER'
targetdb2 <-gsub(' ','',targetdb2)
print (targetdb2)


In [ ]:
# Create SQL command

sql1 <- paste('
WITH distinct_pi AS (
  SELECT DISTINCT person_id, 
  FROM ', targetdb1, '.tbl_CiC
)
SELECT distinct a.person_id, b.gender_source_value
FROM distinct_pi a
JOIN ', targetdb2,'.person b ON b.person_id = a.person_id
LIMIT 4000;
', sep = "")

#This runs it
tb3 <- bq_project_query(project_id, sql1)

#This loads  it into an R data frame
table <- bq_table_download(tb3)
#This displays it
table


In [ ]:
schema <- list(
  list("person_id", "INTEGER"),
  list("gender_source_value", "STRING")
)

# Reverse the column order
reverse <- table[, rev(colnames(table))]
bq_table_upload("yhcr-prd-phm-bia-core.CB_2353.CiC_gender", reverse, schema = schema)